In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame, Series

In [ ]:
np.random.seed(20171121)

n0, mu0, variance0 = 20, [10, 11], 20